# LangGraph

In [1]:
import os
import operator
from typing import Annotated, Any

from pydantic import BaseModel, Field

from langgraph.graph import StateGraph
from langchain_openai import ChatOpenAI

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

from langchain_core.runnables import ConfigurableField

from langgraph.graph import END

from langfuse.callback import CallbackHandler

In [2]:
LANGFUSE_HOST = os.getenv("LANGFUSE_HOST")
SECRET_KEY = os.getenv("SECRET_KEY")
PUBLIC_KEY = os.getenv("PUBLIC_KEY")

langfuse_handler = CallbackHandler(
    public_key=PUBLIC_KEY,
    secret_key=SECRET_KEY,
    host=LANGFUSE_HOST,
)


## Ollamaを使ったLanggraphのサンプルコード

In [3]:
class State(BaseModel):
    """ステートクラス.

    Args:
        BaseModel (_type_): _description_
    """

    query: str = Field(..., description="ユーザーからの質問")
    current_role: str = Field(default="", description="選定された解答ロール")
    # operator.addは2つの値を加算するための関数
    # ステート更新時にaddオペレーションにより、リストに要素が追加される。リストの足し算と同義
    messages: Annotated[list[str], operator.add] = Field(default=[], description="解答履歴")
    current_judge: bool = Field(default=False, description="品質チェックの結果")
    judgment_reason: str = Field(default="", description="品質チェックの判定理由")

In [4]:
ROLES = {
    "1": {
        "name": "一般知識のエキスパート",
        "description": "幅広い分野の一般的な質問に答える",
        "details": "幅広い分野の一般的な質問に対して、正確でわかりやすい回答を提供してください。",
    },
    "2": {
        "name": "生成AI製品エキスパート",
        "description": "生成AIや関連製品、技術に関する専門的な質問に答える",
        "details": "生成AIや関連製品、技術に関する専門的な質問に対して、最新の情報と深い洞察を提供してください。",
    },
    "3": {
        "name": "カウンセラー",
        "description": "個人的な悩みや心理的な問題に対してサポートを提供する",
        "details": "個人的な悩みや心理的な問題に対して、共感的で支援的な回答を提供し、可能であれば適切なアドバイスも行ってください。",
    },
}

In [5]:
OLLAMA_HOST = os.getenv("OLLAMA_HOST")
llm = ChatOpenAI(model="gemma3:12b", temperature=0.8, openai_api_base=f"{OLLAMA_HOST}/v1", openai_api_key="dummy")
llm = llm.configurable_fields(max_tokens=ConfigurableField(id="max_tokens"))

In [6]:
def selection_node(state: State) -> dict[str, Any]:
    query = state.query
    role_options = "\n".join([f"{k}. {v['name']}: {v['description']}" for k, v in ROLES.items()])
    prompt = ChatPromptTemplate.from_template(
        """質問を分析し、最も適切な回答担当ロールを選択してください。

選択肢:
{role_options}

回答は選択肢の番号（1、2、または3）のみを返してください。

質問: {query}
""".strip()
    )

    # 選択肢の番号のみを返すことを期待したいため、max_tokensの値を1に変更
    chain = prompt | llm.with_config(configurable=dict(max_tokens=1)) | StrOutputParser()
    role_number = chain.invoke({"role_options": role_options, "query": query}, config={"callbacks": [langfuse_handler]})

    selected_role = ROLES[(role_number.strip())]["name"]

    return {"current_role": selected_role}

In [7]:
def answering_node(state: State) -> dict[str, Any]:
    query = state.query
    role = state.current_role
    role_details = "\n".join([f"- {v['name']}: {v['details']}" for v in ROLES.values()])

    prompt = ChatPromptTemplate.from_template(
        """あなたは{role}として回答してください。以下の質問に対して、あなたの役割に基づいた適切な回答を提供してください。

役割の詳細:
{role_details}

質問: {query}

回答:""".strip()
    )

    chain = prompt | llm | StrOutputParser()
    answer = chain.invoke(
        {"role": role, "role_details": role_details, "query": query}, config={"callbacks": [langfuse_handler]}
    )

    return {"messages": [answer]}

In [8]:
class Judgement(BaseModel):
    reason: str = Field(default="", description="判定理由")
    judge: bool = Field(default=False, description="判定結果")


def check_node(state: State) -> dict[str, Any]:
    query = state.query
    answer = state.messages[-1]

    prompt = ChatPromptTemplate.from_template(
        """以下の回答の品質をチェックし、問題がある場合は'False'、問題がない場合は'True'を回答してください。また、その判定理由も説明してください。

ユーザーからの質問: {query}
回答: {answer}
""".strip()
    )

    chain = prompt | llm.with_structured_output(Judgement)

    result: Judgement = chain.invoke({"query": query, "answer": answer}, config={"callbacks": [langfuse_handler]})

    return {
        "current_judge": result.judge,
        "judgment_reason": result.reason,
    }

In [9]:
workflow = StateGraph(State)

In [10]:
workflow.add_node("selection", selection_node)
workflow.add_node("answering", answering_node)
workflow.add_node("check", check_node)

# selectionノードから処理を開始
workflow.set_entry_point("selection")

# エッジの接続
workflow.add_edge("selection", "answering")
workflow.add_edge("answering", "check")

In [11]:
# 条件付きエッジの定義
workflow.add_conditional_edges("check", lambda state: state.current_judge, {True: END, False: "selection"})

In [12]:
compiled = workflow.compile()

In [13]:
initial_state = State(query="生成AIについて教えてください")
result = compiled.invoke(initial_state, config={"callbacks": [langfuse_handler]})
print(result["messages"])

['はい、生成AIについてですね。生成AI製品エキスパートとして、最新の情報と深い洞察を交えながら、分かりやすくご説明します。\n\n**1. 生成AIとは何か？**\n\n生成AIとは、既存のデータから学習し、新しいコンテンツを生成するAI技術のことです。従来のAIは、データに基づいて予測や分類を行うことが中心でしたが、生成AIはテキスト、画像、音声、動画など、さまざまな形式のコンテンツを「創り出す」ことができます。\n\n**例:**\n\n*   **文章生成:** 自然な文章を書く (小説、詩、メール、レポートなど)\n*   **画像生成:** テキスト指示に基づいて画像を生成する (風景、人物、抽象画など)\n*   **音楽生成:** 既存の音楽スタイルを模倣したり、新しい音楽を作曲する\n*   **動画生成:** テキストや画像から動画を生成する\n*   **コード生成:** プログラミングコードを生成する\n\n**2. 主なモデルと技術**\n\n生成AIを支える主要な技術とモデルをいくつかご紹介します。\n\n*   **Transformer:** 現在の多くの生成AIモデルの基盤となる、自然言語処理において優れた性能を発揮する深層学習アーキテクチャです。Attentionメカニズムが特徴で、文章の文脈を理解し、より自然な文章を生成できます。\n*   **GPT (Generative Pre-trained Transformer):** OpenAIが開発した大規模言語モデルで、GPT-3、GPT-4などが有名です。テキスト生成、翻訳、要約、質疑応答など、幅広いタスクに活用されています。\n*   **DALL-E:** OpenAIが開発した画像生成AIモデルで、テキストによる指示に基づいてユニークな画像を生成します。\n*   **Stable Diffusion:** 画像生成AIモデルで、オープンソースであり、商用利用も可能です。\n*   **Midjourney:** Discord上で動作する画像生成AIで、芸術的な表現に優れています。\n*   **LoRA (Low-Rank Adaptation):** 大規模言語モデルを特定のタスクやスタイルに微調整する効率的な手法です。\n\n**3. 生成AIの活用事例**